In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '142RKXC9FW3PDC9F',
  'HostId': '4CCqls1QXeS0gYDT2zfvJBbEg0AVOTwSoiUOS2+K+TR+ENLcZkI4kFyhMjrWGK6mF0IvUZj5m1mtohejtx7dEMXazF19L7Kl',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '4CCqls1QXeS0gYDT2zfvJBbEg0AVOTwSoiUOS2+K+TR+ENLcZkI4kFyhMjrWGK6mF0IvUZj5m1mtohejtx7dEMXazF19L7Kl',
   'x-amz-request-id': '142RKXC9FW3PDC9F',
   'date': 'Thu, 24 Apr 2025 17:44:33 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'cities-test-sandbox',
   'CreationDate': datetime.datetime(2025, 4, 18, 19, 10, 

In [ ]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-04-24 12:44:36,407][INFO    ][coiled] Fetching latest package priorities...
[2025-04-24 12:44:36,408][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-04-24 12:44:37,378][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-04-24 12:44:37,389][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-04-24 12:44:39,069][INFO    ][coiled] Running pip check...
[2025-04-24 12:44:41,119][INFO    ][coiled] Validating environment...
[2025-04-24 12:44:43,933][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-04-24 12:44:44,380][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-04-24 12:44:44,382][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-04-24 12:44:45,343][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-syvrn.dask.host/DauPWWfNSiNnSF4z/status


Task exception was never retrieved
future: <Task finished name='Task-1965' coro=<Client._gather.<locals>.wait() done, defined at /Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:2394> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py", line 2403, in wait
    raise AllExit()
distributed.client.AllExit


In [6]:
import dask_geopandas as dgpd
import dask.dataframe as dd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
from metrics_groupby import metrics

from pre_processing import *
from auxiliary_functions import *
from standardize_metrics import *

YOUR_NAME = 'sara'
grid_size = 200


In [ ]:
from pre_processing import *

import time
start_time = time.time()  # Start the timer

%autoreload
#from main import *


def preprocess_all_cities(city_list):
    #delayed_jobs = [delayed(calculate_building_distances_to_roads)(city) for city in city_list]
    #delayed_jobs.append([delayed(produce_blocks)(city) for city in city_list])
    delayed_jobs = []
    delayed_jobs.append([calculate_building_distances_to_roads(city) for city in city_list])
    delayed_jobs.append([produce_blocks(city,YOUR_NAME,grid_size) for city in city_list])
    results = compute(*delayed_jobs)
    return results

preprocess_all_cities(["Belo_Horizonte", "Medellin"]) 
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 904.61 seconds.


In [ ]:

from pre_processing import *

import time
start_time = time.time()  # Start the timer


city_list = ["Belo_Horizonte", "Medellin"]
#["Abidjan", "Accra", "Nairobi", "Bamako", "Belo_Horizonte", "Bogota", "Campinas", "Cape_Town", "Lagos","Luanda", "Nairobi", "Medellin"])

delayed_jobs = []
delayed_jobs.append([produce_azimuths(city, YOUR_NAME, grid_size) for city in city_list])
results = compute(*delayed_jobs)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 133.54 seconds.


In [16]:
@delayed
def building_and_intersection_metrics(city_name):
    grid_cell_count = 0
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    # Get EPSG
    epsg = get_epsg(city_name).compute()
    # Load grid
    grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    grid['cell_area'] = grid.geometry.area

    cells = grid.index.size
    grid_cell_count += cells

    # Load buildings and perform relevant calculations on it
    buildings = load_dataset(paths['buildings'], epsg=epsg)#.compute()
    buildings['area'] = buildings.geometry.area
    joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
    counts_buildings = joined_buildings.groupby('index_right').size()
    grid['n_buildings'] = grid.index.map(counts_buildings.astype(int))#.fillna(0.).astype(int)
    built_area_buildings = joined_buildings.groupby('index_right')['area'].sum()
    grid['built_area'] = grid.index.map(built_area_buildings).astype(float)#.fillna(0.).astype(float)

    #total_buildings = row_count(buildings).compute()
    #print(total_buildings)
    # Load roads
    roads = load_dataset(paths['roads'], epsg=epsg)#.compute()
    included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']

    def highway_filter(highway_value):
        # If highway_value is missing, return False
        if pd.isna(highway_value):
            return False
        # Split the string by commas, and strip any whitespace from each part
        types = [part.strip() for part in highway_value.split(',')]
        # Return True if any of the types is in our included list
        return any(t in included_road_types for t in types)

    # Now filter the roads GeoDataFrame:
    roads = roads[roads['highway'].apply(highway_filter)]
    #roads['highway'][roads['highway'].isin(included_road_types)]
    
    #road_union = roads.unary_union.compute()
    #roads = roads.compute()

    # Load intersections
    intersections = load_dataset(paths['intersections'], epsg=epsg)#.compute()

    all_valid_intersections = intersections[intersections.street_count >= 2]
    intersections_3plus = intersections[intersections.street_count >= 3]
    intersections_4way = intersections[intersections.street_count == 4]

    grid['cell_area_km2'] = grid['cell_area']/1000000.


    roads_grid_joined = dgpd.sjoin(roads, grid, predicate='intersects')

    roads_grid_joined = roads_grid_joined.merge(
        grid[['geometry']],
        left_on='index_right',
        right_index=True,
        suffixes=('', '_grid')
    )

    # Compute the intersection (i.e. the "cookie-cutter") road segment within the grid cell for each row:
    def compute_intersection_length(row):
        try:
            inter_geom = row['geometry'].intersection(row['geometry_grid'])
            return inter_geom.length
        except Exception:
            return 0.0

    roads_grid_joined['road_length_in_cell'] = roads_grid_joined.apply(compute_intersection_length, axis=1)

    # Now sum the intersection lengths by grid cell index:
    road_length_km = roads_grid_joined.groupby('index_right')['road_length_in_cell'].sum() / 1000.    

    #road_length_km = roads_grid_joined.groupby('index_right')['length'].sum()/1000.
    grid['road_length'] = grid.index.map(road_length_km).astype(float)#.fillna(0.).astype(float)
    grid['has_roads'] = grid['road_length'].fillna(0) > 0

    joined_all_valid_intersections = dgpd.sjoin(all_valid_intersections, grid, predicate='intersects')
    counts_all_valid_intersections = joined_all_valid_intersections.groupby('index_right').size()
    grid['n_intersections'] = grid.index.map(counts_all_valid_intersections).astype(float)
    grid['has_intersections'] = grid['n_intersections'].fillna(0)> 0

    joined_intersections_3plus = dgpd.sjoin(intersections_3plus, grid, predicate='intersects')
    counts_intersections_3plus = joined_intersections_3plus.groupby('index_right').size()
    grid['intersections_3plus'] = grid.index.map(counts_intersections_3plus).astype(float)#.fillna(0.).astype(float)

    joined_intersections_4way = dgpd.sjoin(intersections_4way, grid, predicate='intersects')
    counts_intersections_4way = joined_intersections_4way.groupby('index_right').size()
    grid['intersections_4way'] = grid.index.map(counts_intersections_4way).astype(float)#.fillna(0.).astype(float) # OJO: NEED TO CHANGE NA


    grid['m3_raw'] = grid['road_length']/grid['cell_area_km2']
    grid['m3_raw'] = grid['m3_raw']#.fillna(0.)
    grid['m3_std'] = grid['m3_raw'].map_partitions(standardize_metric_3, meta=('m3', 'float64'))

    grid['m4_raw'] = grid['intersections_4way'] / grid['intersections_3plus']
    grid['m4_raw'] = grid['m4_raw'].mask((grid['m4_raw'].isna()) & (grid['road_length'] > 0), 0.)
    avg_m4 = grid['m4_raw'].mean().compute()
    grid['m4_raw'] = grid['m4_raw']#.fillna(avg_m4)
    grid['m4_std'] = grid['m4_raw'].map_partitions(standardize_metric_4, meta=('m4', 'float64'))


    grid['m5_raw'] =  (1000.**2)*(grid['n_intersections']/grid['cell_area']) #make sure this is equivalent to the meter calculation
    grid['m5_raw'] = grid['m5_raw']#.fillna(0.)
    # wherever has_roads is True AND m5_raw is NA, set it to 0
    grid['m5_raw'] = grid['m5_raw'].mask(
        grid['has_roads'] & grid['m5_raw'].isna(),
        0.
    )
    grid['m5_std'] = grid['m5_raw'].map_partitions(standardize_metric_5, meta=('m5', 'float64'))

    
    grid['m11_raw'] = 1.0*grid['n_buildings'] / grid['cell_area_km2'] # Building density
    grid['m11_raw'] = grid['m11_raw']#.fillna(0.)
    grid['m11_std'] = grid['m11_raw'].map_partitions(standardize_metric_11, meta=('m11', 'float64'))

    grid['m12_raw'] = grid['built_area'] / grid['cell_area'] # Built area share
    grid['m12_raw'] = grid['m12_raw']#.fillna(0.)
    grid['m12_std'] = grid['m12_raw'].map_partitions(standardize_metric_12, meta=('m12', 'float64'))

    grid['m13_raw'] = grid['built_area'] / grid['n_buildings'] # Average building area
    grid['m13_raw'] = grid['m13_raw']#.fillna(0.)
    grid['m13_std'] = grid['m13_raw'].map_partitions(standardize_metric_13, meta=('m13', 'float64'))

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_metrics_3_4_5_11_12_13_grid_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)
    return grid_cell_count, path


@delayed
def building_distance_metrics(city_name):

     grid_cell_count = 0
     paths = {
         'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
         'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
         'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
         'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
         'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
     }
     # Get EPSG
     epsg = get_epsg(city_name).compute()
     # Load grid
     grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
     if 'geom' in grid.columns:
         grid = grid.drop(columns=['geom'])
     
     buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)#.compute()
     buildings['distance_to_nearest_road'] = buildings['distance_to_nearest_road'].astype(float)
     buildings['area'] = buildings.geometry.area
     joined_buildings = dgpd.sjoin(buildings, grid, predicate='intersects')  
     counts_buildings = joined_buildings.groupby('index_right').size()
     grid['n_buildings'] = grid.index.map(counts_buildings).fillna(0).astype(int)
     grid['has_buildings'] = grid['n_buildings']    > 0
     average_distance = joined_buildings.groupby('index_right')['distance_to_nearest_road'].mean()
     grid['average_distance_nearest_building'] = grid.index.map(average_distance).astype(float)#fillna(np.mean(average_distance)).astype(float)
    
    
     buildings_closer_than_20m = buildings[buildings['distance_to_nearest_road'] <= 20]
     joined_buildings_closer_than_20m = dgpd.sjoin(buildings_closer_than_20m, grid, predicate='intersects') 
     n_buildings_closer_than_20m = joined_buildings_closer_than_20m.groupby('index_right').size()
     grid['n_buildings_closer_than_20m'] = grid.index.map(n_buildings_closer_than_20m).astype(float)#.fillna(np.mean(n_buildings_closer_than_20m))
     grid = grid.assign(
    n_buildings_closer_than_20m = grid['n_buildings_closer_than_20m'].mask(
        (grid['n_buildings'] > 0) & (grid['n_buildings_closer_than_20m'].isna()),
        0))
     grid = grid.assign(
         m1_raw = grid['n_buildings_closer_than_20m'] / grid['n_buildings']
         )
     grid['m1_raw'] = grid['m1_raw']#.fillna(grid['m1_raw'].mean())#grid['n_buildings_closer_than_20m'] / grid['n_buildings']
     grid['m1_std'] = grid['m1_raw'].map_partitions(standardize_metric_1, meta=('m1', 'float64'))
     grid['m2_raw'] = grid['average_distance_nearest_building']
     grid['m2_raw'] = grid['m2_raw']#.fillna(grid['m2_raw'].mean())
     grid['m2_std'] = grid['m2_raw'].map_partitions(standardize_metric_2, meta=('m2', 'float64'))
     
     path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
    
     if 'geom' in grid.columns:
         grid = grid.drop(columns='geom')
    
     grid.to_parquet(path)



@delayed
def compute_m6_m7_m8(city_name):
    """
    Computes:
    - M6: KL divergence (building orientation)
    - M7: Average block width
    - M8: Building density ratio (inner vs. outer buffer)
    """

    epsilon = 0.001
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
        'buildings_with_distances_azimuths': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet',
        'buildings_to_blocks':f'{BLOCKS_PATH}/{city_name}/{city_name}_buildings_to_blocks_{YOUR_NAME}.geoparquet'
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'], epsg=epsg).persist()
    buildings = load_dataset(paths['buildings_with_distances_azimuths'], epsg=epsg).persist()
    buildings['azimuth'] = buildings['azimuth'].map_partitions(pd.to_numeric, errors='coerce')


    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    
    blocks['block_id'] = blocks.index
    blocks['epsilon_buffer'] = blocks['geometry'].buffer(-(1.- epsilon) * blocks['max_radius'])
    blocks['width_buffer'] = blocks['geometry'].buffer(-0.2 * blocks['max_radius'])

    buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects').persist() #,how='right'
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'epsilon_buffer','width_buffer','azimuth']]
    buildings_blocks = buildings_blocks.set_index('block_id').repartition(npartitions=4)

    block_grid_overlap = compute_block_grid_weights(blocks, grid)
    block_grid_overlap = block_grid_overlap.compute()

    # Metric 6
    kl_df = compute_block_kl_metrics(buildings_blocks)
    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap)
 
    # Metric 7
    block_grid_overlap['weighted_max_radius'] = (
        block_grid_overlap['max_radius'] * block_grid_overlap['area_weight']
    )

    grid_m7 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_max_radius=('weighted_max_radius', 'sum'),
        total_weight=('area_weight', 'sum')
    )
    grid_m7['m7'] = grid_m7['total_weighted_max_radius'] / grid_m7['total_weight']

    # Metric 8
    width_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='width_buffer')
    epsilon_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='epsilon_buffer')
    clipped_buildings_area_to_buffer_ratio = epsilon_buffer_ratios / width_buffer_ratios
    clipped_buildings_area_to_buffer_ratio = clipped_buildings_area_to_buffer_ratio.replace([np.inf, -np.inf], np.nan)#.fillna(999)
    ratio_df = clipped_buildings_area_to_buffer_ratio.to_frame(name='m8')
    blocks_with_m8 = blocks.merge(ratio_df, left_on='block_id', right_index=True, how='left').compute()
    block_grid_overlap = block_grid_overlap.merge(blocks_with_m8, how='left',left_on='block_id',right_index=True)
    block_grid_overlap['weighted_m8'] = (
        block_grid_overlap['m8'] * block_grid_overlap['area_weight']
    )
    grid_m8 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_m8=('weighted_m8', 'sum'),
        total_weight=('area_weight', 'sum')
    )
    grid_m8['m8'] = grid_m8['total_weighted_m8'] / grid_m8['total_weight']

    # Merge all metrics
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m7[['m7']], left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m8[['m8']], left_index=True, right_index=True, how='left')

    '''
    # Fill NaNs
    '''

    grid['m6_raw'] = grid['m6']#.fillna(grid['m6'].mean())
    grid['m6_std'] = grid['m6_raw'].map_partitions(standardize_metric_6, meta=('m6', 'float64'))


    grid['m7_raw'] = grid['m7']#.fillna(grid['m7'].max())
    grid['m7_std'] = grid['m7_raw'].map_partitions(standardize_metric_7, meta=('m7', 'float64'))
    grid['m8_raw'] = grid['m8']#.fillna(grid['m8'].max())
    grid['m8_std'] = grid['m8_raw'].map_partitions(standardize_metric_8, meta=('m8', 'float64'))
    
    grid = grid.drop(columns=['m6','m7','m8'])
    # Save Output
    grid = grid.compute()  
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    grid.to_parquet(path)
    
    #path = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_with_m8_{YOUR_NAME}.geoparquet'#f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    #blocks_with_m8.to_parquet(path)
    return  path

def partition_tortuosity_clipped(df):
    # df.geometry is the clipped segment
    seg = df.geometry
    L = seg.length.values            # true path length
    b = seg.bounds                   # DataFrame with minx,miny,maxx,maxy
    Dx = (b["maxx"] - b["minx"]).values
    Dy = (b["maxy"] - b["miny"]).values
    D  = np.hypot(Dx, Dy)            # straight‐line distance
    S  = np.clip(D / L, 0, 1)        # straightness in [0,1]
    W  = L * S                       # length‐weighted straightness

    return pd.DataFrame({
        "index_right": df["index_right"].values,
        "wt":           W,
        "length":      L
    }, index=df.index)

def overlay_partition(roads_part, grid_small):
    # roads_part: a pandas GeoDataFrame partition of full roads GDF
    # grid_small: a pandas GeoDataFrame of all grid cells with an index_right column
    return gpd.overlay(roads_part, grid_small, how="intersection")


@delayed
def metrics_roads_intersections(city_name):

    paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }

    # LOAD
    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg).persist()
    roads = load_dataset(paths['roads'], epsg=epsg).persist()
    intersections = load_dataset(paths['intersections'], epsg=epsg).compute()


    included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']
    roads['highway'][roads['highway'].isin(included_road_types)]
    

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])


    
    # Prep for metric 10
    intersections['osmid'] = intersections['osmid'].astype(int)
    intersection_angles = compute_intersection_angles(roads, intersections)
    street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
    intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
    intersection_angle_mapping = intersection_angle_mapping.compute()  

    intersections_with_angles_metric = intersections.merge(
        intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
    )

    joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
    average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()
    

    # Prep for metric 9
    
    roads_simple = roads[['geometry']]
    
    grid_small = (
        grid
        .reset_index()[["index","geometry"]]
        .rename(columns={"index":"index_right"})
    )
    
    # inside metrics_roads_intersections, before overlay:
    overlay_meta = gpd.GeoDataFrame(
        {
            "index_right": pd.Series(dtype="int64"),
            "geometry":    gpd.GeoSeries(dtype="geometry")
        },
        geometry="geometry"
    )

    roads_cells = roads_simple.map_partitions(
        overlay_partition,
        grid_small,
        meta=overlay_meta
    ).persist()

    # roads_cells now has:
    #  - geometry   = clipped road piece
    #  - index_right = the cell it belongs to

    # 3) compute wt & length in one pass per partition
    out = roads_cells.map_partitions(
        partition_tortuosity_clipped,   
        meta=pd.DataFrame({
            "index_right": pd.Series(dtype="int64"),
            "wt":           pd.Series(dtype="float64"),
            "length":      pd.Series(dtype="float64")
        })
    ).persist()

    # 4) aggregate back into grid
    agg = out.groupby("index_right").agg(
        total_len=("length","sum"),
        sum_wt   =("wt",    "sum")
    )
    grid["m9_raw"] = grid.index.map(agg["sum_wt"]/agg["total_len"]).astype(float)
    grid["m9_std"] = grid["m9_raw"].map_partitions(
        standardize_metric_9, meta=("m9","float64")
    )
    

    '''
    roads_with_tortuosity = calculate_tortuosity(roads.compute(), intersections)
    joined_tortuosity_grid = dgpd.sjoin(roads_with_tortuosity.set_geometry('road_geometry'), grid, predicate="intersects")
    average_tortuosity = joined_tortuosity_grid.groupby('index_right')['tortuosity'].mean()


    grid['m9_raw'] = grid.index.map(average_tortuosity).astype(float)#.fillna(0.).astype(float)
    grid['m9_std'] = grid['m9_raw'].map_partitions(standardize_metric_9, meta=('m9', 'float64'))
    '''
    
    grid['m10_raw'] = grid.index.map(average_angle_between_roads).astype(float)#.fillna(np.mean(average_angle_between_roads)).astype(float)
    grid['m10_std'] = grid['m10_raw'].map_partitions(standardize_metric_10, meta=('m10', 'float64'))
    
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)

    return path

In [17]:
import time
from dask import compute

start_time = time.time()  # Start the timer

cities = ["Medellin"]#['Nairobi']#"Belo_Horizonte", 
cities = [city.replace(' ', '_') for city in cities]

tasks = []
for city in cities:
    tasks.append(building_and_intersection_metrics(city))
    #tasks.append(building_distance_metrics(city))
    #tasks.append(compute_m6_m7_m8(city))
    #tasks.append(metrics_roads_intersections(city))

results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

Tasks completed in 291.58 seconds.


In [9]:
import dask_geopandas as dgpd
import pandas as pd

def consolidate_irregularity_index(city_name):
    # Define file paths
    path_metrics_most = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_metrics_3_4_5_11_12_13_grid_{YOUR_NAME}.geoparquet'
    path_metrics_distances = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
    path_metrics_blocks = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    path_metrics_intersections = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'
    
    # Read each file as a Dask GeoDataFrame
    grid_most = dgpd.read_parquet(path_metrics_most)
    grid_distances = dgpd.read_parquet(path_metrics_distances)[['m1_std','m1_raw','m2_std','m2_raw','has_buildings']]
    grid_blocks = dgpd.read_parquet(path_metrics_blocks)[['m6_std','m6_raw','m7_std','m7_raw','m8_std','m8_raw']]
    grid_intersections = dgpd.read_parquet(path_metrics_intersections)[['m9_std','m9_raw','m10_std','m10_raw']]
    
    # Drop duplicate geometry columns from non-base grids.
    # Explicitly reassign so the changes persist.
    grid_distances = grid_distances.drop(columns=['geometry'], errors='ignore')
    grid_blocks    = grid_blocks.drop(columns=['geometry'], errors='ignore')
    grid_intersections = grid_intersections.drop(columns=['geometry'], errors='ignore')
    
    # Merge the files on the index (left join on grid_most)
    grid_combined = grid_most.merge(grid_distances, left_index=True, right_index=True, how='left')
    grid_combined = grid_combined.merge(grid_blocks, left_index=True, right_index=True, how='left')
    grid_combined = grid_combined.merge(grid_intersections, left_index=True, right_index=True, how='left')
    
    # Process metrics: note that here we assume that standardized metric columns already exist
    all_metrics_columns = ['m'+str(x) for x in range(1,14)]
    all_metrics_columns_final = ['m'+str(x)+'_final' for x in range(1,14)]
    metrics_standardized_names = [col+'_std' for col in all_metrics_columns]
    
    # Create mask for raw and standardized names, for this to propagate downstream.
    raw_cols = [f'm{i}_raw' for i in range(1,14)]
    grid_combined['has_features'] = grid_combined['has_buildings'] | grid_combined['has_roads'] | grid_combined['has_intersections']
    grid_combined[metrics_standardized_names] = grid_combined[metrics_standardized_names].where(grid_combined['has_features'], np.nan)
    grid_combined[raw_cols] = grid_combined[raw_cols].where(grid_combined['has_features'], np.nan)

    # Create zero-centered copies of the standardized metric columns.
    zero_centered_names_list = [col+'_zero-centered' for col in all_metrics_columns]
    grid_combined[zero_centered_names_list] = grid_combined[metrics_standardized_names].copy() #grid_combined[metrics_standardized_names].copy()
    
    # Center at zero (subtract mean and divide by std)
    grid_combined[zero_centered_names_list] = grid_combined[zero_centered_names_list].apply(
        lambda x: (x - x.mean()) / x.std(), axis=1
    )
    
    # Copy the zero-centered values to the base metric columns
    grid_combined[all_metrics_columns_final] = grid_combined[zero_centered_names_list].copy()
    
    # Rescale metrics to [0,1] using min–max normalization.
    grid_combined[all_metrics_columns_final] = grid_combined[all_metrics_columns_final].apply(
        lambda x: (x - x.min()) / (x.max() - x.min()), axis=1
    )
    
    # Calculate equal-weights irregularity index as the mean across all metrics.
    # (Using axis=1 because we average across columns.)
    grid_combined['regularity_index'] = grid_combined[all_metrics_columns_final].mean(axis=1)

    # Add a column that counts the number of NA values per row among the 13 final metric columns.
    grid_combined['na_count'] = grid_combined[all_metrics_columns_final].isna().sum(axis=1)
    
    
    # Define output path and save the merged grid
    out_path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_all_metrics_combined_{YOUR_NAME}.geoparquet'
    #grid_combined.to_parquet(out_path)
    
    return grid_combined


grid_combined = consolidate_irregularity_index('Medellin')
grid_combined = grid_combined.compute()
#print("Combined grid saved to:", consolidated_path)


/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/dask/dataframe/dask_expr/_collection.py:3243: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta={'m1_zero-centered': 'float64', 'm2_zero-centered': 'float64', 'm3_zero-centered': 'float64', 'm4_zero-centered': 'float64', 'm5_zero-centered': 'float64', 'm6_zero-centered': 'float64', 'm7_zero-centered': 'float64', 'm8_zero-centered': 'float64', 'm9_zero-centered': 'float64', 'm10_zero-centered': 'float64', 'm11_zero-centered': 'float64', 'm12_zero-centered': 'float64', 'm13_zero-centered': 'float64'})

  warnings.warn(meta_warning(meta))
/Users/sarangof/miniconda3/envs/subdivisions2/lib/py

In [10]:
grid_combined.to_parquet('../Medellin_consolidated_index_results.parquet')#regularity_index.describe() #[['m1_raw','m2_raw','regularity_index']]

In [11]:
grid_combined.to_file('../Medellin_consolidated_index_results.shp')

/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_66546/401492889.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_combined.to_file('../Medellin_consolidated_index_results.shp')
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'n_buildings' to 'n_building'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'cell_area_km2' to 'cell_area_'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'road_length' to 'road_lengt'
  ogr_write(
/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'n_intersections' to 'n_intersec'
  ogr_wr

In [ ]:
city_name = 'Nairobi'
# Define file paths
path_metrics_most = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_metrics_3_4_5_11_12_13_grid_{YOUR_NAME}.geoparquet'
path_metrics_distances = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_{YOUR_NAME}_metrics_1_2.geoparquet'
path_metrics_blocks = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
path_metrics_intersections = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'

# Read each file as a Dask GeoDataFrame
grid_most = dgpd.read_parquet(path_metrics_most)
grid_distances = dgpd.read_parquet(path_metrics_distances)[['m1','m1_raw','m2','m2_raw']]
grid_blocks = dgpd.read_parquet(path_metrics_blocks)[['m6','m6_raw','m7','m7_raw','m8','m8_raw']]
grid_intersections = dgpd.read_parquet(path_metrics_intersections)[['m9','m9_raw','m10','m10_raw']]

# Drop duplicate geometry columns from non-base grids.
# Explicitly reassign so the changes persist.
grid_distances = grid_distances.drop(columns=['geometry'], errors='ignore')
grid_blocks    = grid_blocks.drop(columns=['geometry'], errors='ignore')
grid_intersections = grid_intersections.drop(columns=['geometry'], errors='ignore')

# Merge the files on the index (left join on grid_most)
grid_combined = grid_most.merge(grid_distances, left_index=True, right_index=True, how='left')
grid_combined = grid_combined.merge(grid_blocks, left_index=True, right_index=True, how='left')
grid_combined = grid_combined.merge(grid_intersections, left_index=True, right_index=True, how='left')

# Process metrics: note that here we assume that standardized metric columns already exist
all_metrics_columns = ['m'+str(x)+'_raw' for x in range(1,14)]
all_metrics_columns_final = ['m'+str(x)+'_final' for x in range(1,14)]
metrics_standardized_names = all_metrics_columns#= {col: col+'_standardized' for col in all_metrics_columns}

# Create zero-centered copies of the standardized metric columns.
zero_centered_names_list = [col+'_zero-centered' for col in all_metrics_columns]
grid_combined[zero_centered_names_list] = grid_combined[metrics_standardized_names].copy()#grid_combined[list(metrics_standardized_names.values())].copy()

# Center at zero (subtract mean and divide by std)
grid_combined[zero_centered_names_list] = grid_combined[zero_centered_names_list].apply(
    lambda x: (x - x.mean()) / x.std(), axis=1
)

# Copy the zero-centered values to the base metric columns
grid_combined[all_metrics_columns_final] = grid_combined[zero_centered_names_list].copy()

# Rescale metrics to [0,1] using min–max normalization.
grid_combined[all_metrics_columns_final] = grid_combined[all_metrics_columns].apply(
    lambda x: (x - x.min()) / (x.max() - x.min()), axis=1
)

# Calculate equal-weights irregularity index as the mean across all metrics.
# (Using axis=1 because we average across columns.)
grid_combined['regularity_index'] = grid_combined[all_metrics_columns_final].mean(axis=1)

# Add a column that counts the number of NA values per row among the 13 final metric columns.
grid_combined['na_count'] = grid_combined[all_metrics_columns_final].isna().sum(axis=1)


In [14]:
city_name = 'Medellin'

paths = {
'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
}

epsg = get_epsg(city_name).compute()
roads = load_dataset(paths['roads'], epsg=epsg)
intersections = load_dataset(paths['intersections'], epsg=epsg).compute()


included_road_types = ['trunk','motorway','primary','secondary','tertiary','primary_link','secondary_link','tertiary_link','trunk_link','motorway_link','residential','unclassified','road','living_street']

def highway_filter(highway_value):
    # If highway_value is missing, return False
    if pd.isna(highway_value):
        return False
    # Split the string by commas, and strip any whitespace from each part
    types = [part.strip() for part in highway_value.split(',')]
    # Return True if any of the types is in our included list
    return any(t in included_road_types for t in types)

# Now filter the roads GeoDataFrame:
roads = roads[roads['highway'].apply(highway_filter)]

/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/dask/dataframe/dask_expr/_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('highway', 'bool'))

  warnings.warn(meta_warning(meta))


In [ ]:
roads.head()

# remove rows if u and v are not part of the 

,u,v,key,osmid,highway,lanes,name,oneway,ref,reversed,length,geometry,maxspeed,width,access,bridge,tunnel,junction,service,area
0,287291520,4330951724,0,737517930,trunk,3,Doble Calzada Niquia - Hatillo,True,25,False,47.473184,"LINESTRING (447935.599 705576.417, 447888.744 ...",nan,nan,nan,nan,nan,nan,nan,nan
1,287291520,9883236579,0,1026331900,residential,nan,nan,False,nan,False,8.457614,"LINESTRING (447935.599 705576.417, 447933.118 ...",nan,nan,nan,nan,nan,nan,nan,nan
2,287291575,8880939061,0,443200030,residential,nan,nan,True,nan,False,13.300952,"LINESTRING (442274.299 701537.538, 442261.036 ...",nan,nan,nan,nan,nan,nan,nan,nan
3,287291575,4408219206,0,573773888,trunk,3,Doble Calzada Niquia - Hatillo,True,25,False,174.396808,"LINESTRING (442274.299 701537.538, 442241.905 ...",nan,nan,nan,nan,nan,nan,nan,nan
4,287291580,8880939092,0,559197931,unclassified,2,nan,True,nan,False,9.100614,"LINESTRING (441363.446 701333.13, 441357.027 7...",20,7,nan,nan,nan,nan,nan,nan


In [16]:
intersections

,osmid,y,x,street_count,highway,railway,ref,geometry
0,287291520,6.383091,-75.470784,3,nan,nan,nan,POINT (447935.599 705576.417)
1,287291575,6.346507,-75.521937,3,nan,nan,nan,POINT (442274.299 701537.538)
2,287291580,6.344650,-75.530170,3,nan,nan,nan,POINT (441363.446 701333.13)
3,287291588,6.338477,-75.542407,3,nan,nan,nan,POINT (440009.309 700652.169)
4,287291691,6.341378,-75.536104,3,nan,nan,nan,POINT (440706.811 700972.124)
...,...,...,...,...,...,...,...,...
66120,12746001862,6.245379,-75.549317,3,nan,nan,nan,POINT (439234.187 690361.064)
66121,12746001866,6.245122,-75.549496,3,nan,nan,nan,POINT (439214.422 690332.685)
66122,12746001871,6.245466,-75.549690,3,nan,nan,nan,POINT (439192.945 690370.791)
66123,12746001872,6.245563,-75.549142,1,nan,nan,nan,POINT (439253.59 690381.429)


In [ ]:
'''

import dask.dataframe as dd
from auxiliary_functions import load_dataset
import dask_geopandas as dgpd
import geopandas as gpd
import pandas as pd
from dask import delayed, compute
from citywide_calculation import get_utm_crs
from dask.diagnostics import ProgressBar
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
from scipy.optimize import fminbound, minimize
from metrics_groupby import metrics
import numpy as np



YOUR_NAME = 'sara'
grid_size = 200

# ------------------------------------------------------------------------------
# 1. Load common datasets (for metrics m1, m2, m3, m4, m5, m9, m10, m11, m12, m13)
# ------------------------------------------------------------------------------
@delayed
def load_common_datasets(city_name):
    epsg = get_epsg(city_name).compute()
    
    grid_path = f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet'
    grid = load_dataset(grid_path, epsg=epsg)
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    grid['cell_area'] = grid.geometry.area
    
    buildings_path = f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet'
    buildings = load_dataset(buildings_path, epsg=epsg)
    
    roads_path = f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet'
    roads = load_dataset(roads_path, epsg=epsg)
    
    intersections_path = f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    intersections = load_dataset(intersections_path, epsg=epsg)
    
    bld_with_dist_path = f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet'
    buildings_with_dist = load_dataset(bld_with_dist_path, epsg=epsg)
    
    return {
         'grid': grid,
         'buildings': buildings,
         'buildings_with_dist': buildings_with_dist,
         'roads': roads,
         'intersections': intersections,
         'epsg': epsg,
    }

# ------------------------------------------------------------------------------
# 2. Metric functions using the common datasets
# ------------------------------------------------------------------------------

# Metric 1: Ratio of buildings within 20 m to total buildings
@delayed
def compute_metric_m1(city_name, datasets):
    grid = datasets['grid']
    buildings = datasets['buildings_with_dist']
    # Ensure distances are floats
    buildings['distance_to_nearest_road'] = buildings['distance_to_nearest_road'].astype(float)
    
    joined = dgpd.sjoin(buildings, grid, predicate='intersects')
    total_buildings = joined.groupby('index_right').size()
    grid = grid.assign(n_buildings=grid.index.map(total_buildings).fillna(0).astype(int))
    
    buildings_close = buildings[buildings['distance_to_nearest_road'] <= 20]
    joined_close = dgpd.sjoin(buildings_close, grid, predicate='intersects')
    count_close = joined_close.groupby('index_right').size()
    
    m1 = grid.index.map(count_close).fillna(0).astype(float) / grid['n_buildings']
    return m1

# Metric 2: Average distance to the nearest road
@delayed
def compute_metric_m2(city_name, datasets):
    grid = datasets['grid']
    buildings = datasets['buildings_with_dist']
    joined = dgpd.sjoin(buildings, grid, predicate='intersects')
    avg_distance = joined.groupby('index_right')['distance_to_nearest_road'].mean()
    m2 = grid.index.map(avg_distance).fillna(0).astype(float)
    return m2

# Metric 3: Road length density (road length per cell area in km²)
@delayed
def compute_metric_m3(city_name, datasets):
    grid = datasets['grid']
    roads = datasets['roads']
    grid = grid.assign(cell_area_km2=grid['cell_area'] / 1e6)
    roads_joined = dgpd.sjoin(roads, grid, predicate='within')
    road_length_km = roads_joined.groupby('index_right')['length'].sum() / 1000.
    m3 = grid.index.map(road_length_km).fillna(0).astype(float) / grid['cell_area_km2']
    return m3

# Metric 4: Ratio of 4-way intersections to intersections with 3+ roads
@delayed
def compute_metric_m4(city_name, datasets):
    grid = datasets['grid']
    intersections = datasets['intersections'].compute()  # force computation if needed
    intersections_3plus = intersections[intersections.street_count >= 3]
    intersections_4way = intersections[intersections.street_count == 4]
    
    joined_3plus = dgpd.sjoin(intersections_3plus, grid, predicate='within')
    count_3plus = joined_3plus.groupby('index_right').size()
    
    joined_4way = dgpd.sjoin(intersections_4way, grid, predicate='within')
    count_4way = joined_4way.groupby('index_right').size()
    
    m4 = grid.index.map(count_4way).fillna(0).astype(float) / grid.index.map(count_3plus).fillna(0)
    return m4

# Metric 5: Intersection density per cell area (using 4-way intersections)
@delayed
def compute_metric_m5(city_name, datasets):
    grid = datasets['grid']
    intersections = datasets['intersections'].compute()
    count_4way = dgpd.sjoin(intersections[intersections.street_count == 4], grid, predicate='within')\
                    .groupby('index_right').size()
    m5 = (1000.**2) * grid.index.map(count_4way).fillna(0).astype(float) / grid['cell_area']
    return m5

# Metric 11: Building density (number of buildings per cell area)
@delayed
def compute_metric_m11(city_name, datasets):
    grid = datasets['grid']
    buildings = datasets['buildings']
    buildings = buildings.assign(area=buildings.geometry.area)
    joined = dgpd.sjoin(buildings, grid, predicate='within')
    count_buildings = joined.groupby('index_right').size()
    m11 = grid.index.map(count_buildings).fillna(0).astype(float) / grid['cell_area']
    return m11

# Metric 12: Built area share (sum of building areas divided by cell area)
@delayed
def compute_metric_m12(city_name, datasets):
    grid = datasets['grid']
    buildings = datasets['buildings']
    buildings = buildings.assign(area=buildings.geometry.area)
    joined = dgpd.sjoin(buildings, grid, predicate='within')
    sum_area = joined.groupby('index_right')['area'].sum()
    m12 = grid.index.map(sum_area).fillna(0).astype(float) / grid['cell_area']
    return m12

# Metric 13: Average building area
@delayed
def compute_metric_m13(city_name, datasets):
    grid = datasets['grid']
    buildings = datasets['buildings']
    buildings = buildings.assign(area=buildings.geometry.area)
    joined = dgpd.sjoin(buildings, grid, predicate='within')
    avg_area = joined.groupby('index_right')['area'].mean()
    m13 = grid.index.map(avg_area).fillna(0).astype(float)
    return m13

# Metric 9: Average road tortuosity
@delayed
def compute_metric_m9(city_name, datasets):
    grid = datasets['grid']
    roads = datasets['roads'].compute()
    intersections = datasets['intersections'].compute()
    # calculate_tortuosity is assumed to be defined elsewhere.
    roads_with_tortuosity = calculate_tortuosity(roads, intersections)
    joined_tortuosity = dgpd.sjoin(roads_with_tortuosity.set_geometry('road_geometry'), grid, predicate="intersects")
    average_tortuosity = joined_tortuosity.groupby('index_right')['tortuosity'].mean()
    m9 = grid.index.map(average_tortuosity).fillna(-999.).astype(float)
    return m9

# Metric 10: Average intersection angle
@delayed
def compute_metric_m10(city_name, datasets):
    grid = datasets['grid']
    intersections = datasets['intersections'].compute()
    intersection_angles = compute_intersection_angles(datasets['roads'], intersections)
    street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
    intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping).compute()
    intersections_with_angles = intersections.merge(
         intersection_angle_mapping.rename("average_angle"),
         left_on="osmid", right_index=True, how="left"
    )
    joined_angles = dgpd.sjoin(intersections_with_angles, grid, predicate="within")
    average_angle = joined_angles.groupby('index_right')['average_angle'].mean()
    m10 = grid.index.map(average_angle).fillna(-999.).astype(float)
    return m10

# ------------------------------------------------------------------------------
# 3. Metric functions for block-based calculations (m6, m7, m8)
# These load additional datasets (blocks, buildings with azimuths) and follow your original logic.
# ------------------------------------------------------------------------------

# Metric 6: KL divergence for building orientation
@delayed
def compute_metric_m6(city_name):
    epsilon = 0.001
    epsg = get_epsg(city_name).compute()
    
    grid_path = f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet'
    grid = load_dataset(grid_path, epsg=epsg)
    if 'geom' in grid.columns:
         grid = grid.drop(columns=['geom'])
    
    blocks_path = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'
    blocks = load_dataset(blocks_path, epsg=epsg).persist()
    
    buildings_az_path = f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet'
    buildings = load_dataset(buildings_az_path, epsg=epsg).persist()
    buildings['azimuth'] = buildings['azimuth'].map_partitions(pd.to_numeric, errors='coerce')
    
    blocks['block_id'] = blocks.index
    blocks['epsilon_buffer'] = blocks['geometry'].buffer(-(1.- epsilon) * blocks['max_radius'])
    blocks['width_buffer'] = blocks['geometry'].buffer(-0.2 * blocks['max_radius'])
    
    buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects').persist()
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'epsilon_buffer', 'width_buffer', 'azimuth']]
    buildings_blocks = buildings_blocks.set_index('block_id').repartition(npartitions=4)
    
    block_grid_overlap = compute_block_grid_weights(blocks, grid)
    block_grid_overlap = block_grid_overlap.compute()
    
    kl_df = compute_block_kl_metrics(buildings_blocks)
    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap)
    
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    m6 = grid['m6'].fillna(0)
    return m6

# Metric 7: Average block width
@delayed
def compute_metric_m7(city_name):
    epsilon = 0.001
    epsg = get_epsg(city_name).compute()
    
    grid_path = f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet'
    grid = load_dataset(grid_path, epsg=epsg)
    if 'geom' in grid.columns:
         grid = grid.drop(columns=['geom'])
    
    blocks_path = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'
    blocks = load_dataset(blocks_path, epsg=epsg).persist()
    
    block_grid_overlap = compute_block_grid_weights(blocks, grid)
    block_grid_overlap = block_grid_overlap.compute()
    block_grid_overlap['weighted_max_radius'] = block_grid_overlap['max_radius'] * block_grid_overlap['area_weight']
    
    grid_m7 = block_grid_overlap.groupby('grid_id').agg(
         total_weighted_max_radius=('weighted_max_radius', 'sum'),
         total_weight=('area_weight', 'sum')
    )
    grid_m7['m7'] = grid_m7['total_weighted_max_radius'] / grid_m7['total_weight']
    
    grid = grid.merge(grid_m7[['m7']], left_index=True, right_index=True, how='left')
    m7 = grid['m7'].fillna(0)
    return m7

# Metric 8: Building density ratio (inner vs. outer buffer)
@delayed
def compute_metric_m8(city_name):
    epsilon = 0.001
    epsg = get_epsg(city_name).compute()
    
    grid_path = f'{GRIDS_PATH}/{city_name}/{city_name}_{grid_size}m_grid.geoparquet'
    grid = load_dataset(grid_path, epsg=epsg)
    if 'geom' in grid.columns:
         grid = grid.drop(columns=['geom'])
    
    blocks_path = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'
    blocks = load_dataset(blocks_path, epsg=epsg).persist()
    
    buildings_az_path = f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances_and_azimuths.geoparquet'
    buildings = load_dataset(buildings_az_path, epsg=epsg).persist()
    buildings['azimuth'] = buildings['azimuth'].map_partitions(pd.to_numeric, errors='coerce')
    
    blocks['block_id'] = blocks.index
    blocks['epsilon_buffer'] = blocks['geometry'].buffer(-(1.- epsilon) * blocks['max_radius'])
    blocks['width_buffer'] = blocks['geometry'].buffer(-0.2 * blocks['max_radius'])
    
    buildings_blocks = dgpd.sjoin(buildings, blocks, predicate='intersects').persist()
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'epsilon_buffer', 'width_buffer', 'azimuth']]
    buildings_blocks = buildings_blocks.set_index('block_id').repartition(npartitions=4)
    
    block_grid_overlap = compute_block_grid_weights(blocks, grid)
    block_grid_overlap = block_grid_overlap.compute()
    
    width_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='width_buffer')
    epsilon_buffer_ratios = buildings_blocks.map_partitions(clip_buildings_by_buffer, buffer_type='epsilon_buffer')
    clipped_buildings_area_to_buffer_ratio = epsilon_buffer_ratios / width_buffer_ratios
    clipped_buildings_area_to_buffer_ratio = clipped_buildings_area_to_buffer_ratio.replace([np.inf, -np.inf], np.nan).fillna(999)
    ratio_df = clipped_buildings_area_to_buffer_ratio.to_frame(name='m8')
    
    blocks_with_m8 = blocks.merge(ratio_df, left_on='block_id', right_index=True, how='left').compute()
    block_grid_overlap = block_grid_overlap.merge(blocks_with_m8, how='left', left_on='block_id', right_index=True)
    block_grid_overlap['weighted_m8'] = block_grid_overlap['m8'] * block_grid_overlap['area_weight']
    
    grid_m8 = block_grid_overlap.groupby('grid_id').agg(
         total_weighted_m8=('weighted_m8', 'sum'),
         total_weight=('area_weight', 'sum')
    )
    grid_m8['m8'] = grid_m8['total_weighted_m8'] / grid_m8['total_weight']
    
    grid = grid.merge(grid_m8[['m8']], left_index=True, right_index=True, how='left')
    m8 = grid['m8'].fillna(-999.)
    return m8

# ------------------------------------------------------------------------------
# 4. Merge all metrics into the grid and save
# ------------------------------------------------------------------------------
def calculate_metrics(city_name):
    # Load common datasets (for m1, m2, m3, m4, m5, m9, m10, m11, m12, m13)
    datasets = load_common_datasets(city_name)
    
    # Compute metrics that use the common datasets
    m1   = compute_metric_m1(city_name, datasets)
    m2   = compute_metric_m2(city_name, datasets)
    m3   = compute_metric_m3(city_name, datasets)
    m4   = compute_metric_m4(city_name, datasets)
    m5   = compute_metric_m5(city_name, datasets)
    m11  = compute_metric_m11(city_name, datasets)
    m12  = compute_metric_m12(city_name, datasets)
    m13  = compute_metric_m13(city_name, datasets)
    m9   = compute_metric_m9(city_name, datasets)
    m10  = compute_metric_m10(city_name, datasets)
    
    # Compute block-based metrics (m6, m7, m8) which load their own extra datasets
    m6   = compute_metric_m6(city_name)
    m7   = compute_metric_m7(city_name)
    m8   = compute_metric_m8(city_name)
    
    @delayed
    def merge_metrics(grid, m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12, m13):
         grid['m1'] = m1
         grid['m2'] = m2
         grid['m3'] = m3
         grid['m4'] = m4
         grid['m5'] = m5
         grid['m6'] = m6
         grid['m7'] = m7
         grid['m8'] = m8
         grid['m9'] = m9
         grid['m10'] = m10
         grid['m11'] = m11
         grid['m12'] = m12
         grid['m13'] = m13
         return grid
    
    final_grid = merge_metrics(datasets['grid'], m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12, m13)
    
    @delayed
    def save_grid(grid, path):
         if 'geom' in grid.columns:
              grid = grid.drop(columns=['geom'])
         grid.to_parquet(path)
         return path
    
    out_path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{grid_size}m_grid_all_metrics_{YOUR_NAME}.geoparquet'
    saved = save_grid(final_grid, out_path)
    result = compute(saved)
    return result[0]

# ------------------------------------------------------------------------------
# 5. Example usage
# ------------------------------------------------------------------------------

if __name__ == "__main__":
    import time
    start_time = time.time()
    city = 'Nairobi'
    city = city.replace(' ', '_')
    final_path = calculate_metrics(city)
    elapsed_time = time.time() - start_time
    print(f"All metrics computed and saved to {final_path}")
    print(f"Tasks completed in {elapsed_time:.2f} seconds.")
'''


In [ ]:
'''
import time
start_time = time.time()
city = 'Nairobi'
city = city.replace(' ', '_')
final_path = calculate_metrics(city)
elapsed_time = time.time() - start_time
print(f"All metrics computed and saved to {final_path}")
print(f"Tasks completed in {elapsed_time:.2f} seconds.")
'''